<a href="https://colab.research.google.com/github/ESSI-Lab-github/dab-pynb/blob/main/HIS-Central.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to the HIS-Central jupyter notebook**

This notebook is used to programmatically access WHOS DAB functionalities, in this case through the OGC OM-JSON based API, documented and available for tests also here: https://whos.geodab.eu/gs-service/om-api

As a prerequisite to obtain programmatically access to WHOS, a token is required. It can be obtained after WHOS user registration here: https://community.wmo.int/en/whos-web-services-and-supported-tools

The first example uses the whos-sensorthingshydro2 view, to obtain records on the HydroServer instance prepared for the workshop where participants can upload their datasets.

The following test portal can also be used to help debug your python notebook: https://whos.geodab.eu/gs-service/search?view=whos-sensorthingshydro2

A series of constraints can be used (e.g. bounding box, temporal extent, observed property, ...) to discover the data. Then it can be accessed and visualized.

In [1]:
import urllib.parse
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import userdata

token = userdata.get('token-whos')
view = 'whos'
#baseURL = 'http://whos.geodab.eu/gs-service/services/essi/token/'
baseURL = 'http://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/'
#baseURL = 'http://boldrini.essi-lab.eu:9090/gs-service/services/essi/token/'



SecretNotFoundError: Secret token-whos does not exist.

In [ ]:
def obfuscate_token(text, token):
    """
    Replaces the token in the given string with '***'.

    Parameters:
        text (str): The string containing the token to obfuscate.
        token (str): The token to obfuscate.

    Returns:
        str: The string with the token replaced by '***'.
    """
    return text.replace(token, "***")

In [ ]:
# clear constraints
bbox = ""
temporal = ""
observedProperty = ""
ontology = ""

In [ ]:
# Algeria example
south = 35.066
west = -1.296
north = 37.466
east = 4.581

view = 'whos-sensorthingshydro2'
bbox = "&west="+str(west)+"&south="+str(south)+"&east="+str(east)+"&north="+str(north)

In [ ]:
# Norway example
south = 60.023
west = 4.253
north = 61.941
east = 8.620

view = 'whos'
bbox = "&west="+str(west)+"&south="+str(south)+"&east="+str(east)+"&north="+str(north)

In [ ]:
# observed property
observedProperty = "&observedProperty="+urllib.parse.quote("Flusso, portata")
ontology="&ontology=whos"

In [ ]:
# observed property
observedProperty = "&observedProperty="+urllib.parse.quote("Flux, discharge")
ontology="&ontology=whos"

In [ ]:
# observed property
observedProperty = "&observedProperty="+urllib.parse.quote("http://hydro.geodab.eu/hydro-ontology/concept/76")
ontology="&ontology=whos"

First we search for stations

In [ ]:
import requests
server = baseURL + token + '/view/' + view + '/om-api/'
url = server+"features?"+observedProperty+ontology+bbox
print("Retrieving "+obfuscate_token(url,token))
response = requests.get(url)

if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    sites = []
    if 'results' not in data:
        print("No results found")
    else:
      for site in data['results']:

          coordinates = site['shape']['coordinates']
          source = next(param['value'] for param in site['parameter'] if param['name'] == 'source')
          identifier = next(param['value'] for param in site['parameter'] if param['name'] == 'identifier')
          name = site['name']
          site_id = site['id']
          if 'relatedParty' in site:
            contact_name = site['relatedParty'][0].get('individualName',None)
            contact_email = site['relatedParty'][0].get('electronicMailAddress',None)
          else:
            contact_name = ""
            contact_email = ""

          sites.append({
              'Name': name,
              'ID': site_id,
              'Coordinates': f"{coordinates[0]}, {coordinates[1]}",
              'Source': source,
              'Identifier': identifier,
              'Contact Name': contact_name,
              'Contact Email': contact_email
          })

      # Create a DataFrame for tabular display
      df = pd.DataFrame(sites)

      styled_df = df.style.set_properties(**{
          'text-align': 'left',  # Align text to the left
          'border': '1px solid black',  # Add borders
          'padding': '5px'  # Add padding
      }).set_table_styles([
          {"selector": "th", "props": [("text-align", "left")]}
      ])

      # Display the table
      display(styled_df)

else:
    print("HTTP GET request failed with status code:", response.status_code)


Then we select a specific feature to list the available observations

In [ ]:
# feature of Algeria

featureId = "15B46866301A3795E9A3C098EDFFB5EE1EB14B64"

In [ ]:
# feature of Norway

featureId = "00513DF7909B1CA51AD5A933ECA6F4D116308B68"

In [ ]:

url = server+"observations?feature="+featureId+observedProperty+ontology+bbox
print("Retrieving "+obfuscate_token(url,token))
response = requests.get(url)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    #print(data)

    observations = []
    for observation in data['member']:
        source = next(param['value'] for param in observation['parameter'] if param['name'] == 'source')
        observed_property_definition = next(param['value'] for param in observation['parameter'] if param['name'] == 'observedPropertyDefinition')
        original_observed_property = next(param['value'] for param in observation['parameter'] if param['name'] == 'originalObservedProperty')
        observed_property = observation['observedProperty']['title']
        phenomenon_time_begin = observation['phenomenonTime']['begin']
        phenomenon_time_end = observation['phenomenonTime']['end']
        feature_of_interest_href = observation['featureOfInterest']['href']
        observation_id = observation['id']
        observation_type = observation['type']
        uom = observation['result']['defaultPointMetadata']['uom']
        interpolation_type = observation['result']['defaultPointMetadata']['interpolationType']['title']

        observations.append({
            'Source': source,
            'Observed Property Definition': observed_property_definition,
            'Original Observed Property': original_observed_property,
            'Observed Property': observed_property,
            'Phenomenon Time Begin': phenomenon_time_begin,
            'Phenomenon Time End': phenomenon_time_end,
            'Feature of Interest Href': feature_of_interest_href,
            'Observation ID': observation_id,
            'Observation Type': observation_type,
            'Unit of Measurement': uom,
            'Interpolation Type': interpolation_type
        })

    # Create a DataFrame for tabular display
    df = pd.DataFrame(observations)

    # Display the table
    display(df)

else:
    print("HTTP GET request failed with status code:", response.status_code)

In [ ]:
# Observation of Algeria
observationId = "urn:uuid:20daeafd-7a98-4332-bfe6-b42e3415ec8f"
beginPosition="&beginPosition="+urllib.parse.quote("1975-01-04T00:00:00Z")
endPosition="&endPosition="+urllib.parse.quote("1975-02-14T00:00:00Z")

In [ ]:
# Observation of Norway
observationId = "urn:uuid:26391b32-63ca-4962-a0c6-08388f1a8670"
beginPosition="&beginPosition="+urllib.parse.quote("2024-09-01T00:00:00Z")
endPosition="&endPosition="+urllib.parse.quote("2024-11-22T00:00:00Z")

In [ ]:
observation="&observationIdentifier="+urllib.parse.quote(observationId)

url = server+"observations?includeData=true"+observation+beginPosition+endPosition
print("Retrieving "+obfuscate_token(url,token))
response = requests.get(url)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Extract the points
    points = data['member'][0]['result']['points']

    # Extract time and values
    times = [datetime.fromisoformat(point['time']['instant'].replace("Z", "+00:00")) for point in points]
    values = [point['value'] for point in points]

    # Plot the data
    plt.figure(figsize=(10, 5))
    plt.plot(times, values, marker='o', linestyle='-', color='b', label='Streamflow')
    plt.title('Streamflow Observations Over Time')
    plt.xlabel('Date')
    plt.ylabel('Value (Cubic Meter)')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


else:
    print("HTTP GET request failed with status code:", response.status_code)